In [3]:
# Import statements
import pandas as pd
import numpy as np
from scipy import stats
import os
import matplotlib.pyplot as plt
from scipy.stats import uniform, truncnorm, randint
from IPython.core.display import display, HTML
from IPython.display import Image
from IPython.core.display import HTML 

from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegressionCV, RidgeClassifierCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter
from imblearn.over_sampling import SMOTE

# Modules defined in the My_Functions.py file
#from My_Functions import *
np.set_printoptions(precision=3)

# Make sure we can see the whole dataset and not a subset
pd.set_option('display.max_rows', 800)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

# Make the notebook fill 100% of browser width.
display(HTML("<style>.container { width:100% !important; }</style>"))

#Setting working directory
#os.chdir("../Data Mining Course Project")

In [105]:
# reading the train dataset
train_data_clean = pd.read_csv('crp_data.csv')

In [106]:
train_data_clean.head()

,household_id,id,rent_monthly,bedrooms_overcrowding,rooms,rooms_overcrowding,has_toilet,has_fridge,tablets_owned_count,males_under_12,males_over_12,males_total,females_under_12,females_over_12,females_total,persons_under_12,persons_over_12,persons_total,total_household,schooling,household_size,walls_brick,walls_socket,walls_prefab,walls_waste,walls_wood,walls_zinc,walls_natural,walls_other,floor_tile,floor_cement,floor_other,floor_natural,floor_na,floor_wood,roof_metal,roof_cement,roof_natural,roof_other,has_cieling,water_inside,water_outside,water_na,elec_public,elec_private,elec_na,elec_coop,toilet_na,toilet_sewer,toilet_septank,toilet_latrine,toilet_other,energ_na,energ_cook_elec,energ_cook_gas,energ_cook_wood,trash_truck,trash_buried,trash_burn,trash_dump,trash_water,trash_other,qualwall_bad,qualwall_reg,qualwall_good,qualroof_bad,qualroof_reg,qualroof_good,qualfloor_bad,qualfloor_reg,qualfloor_good,male,female,ec_under_10,ec_free_union,ec_married,ec_divorced,ec_separated,ec_widow/er,ec_single,pc_house_head,pc_spouse,pc_child,pc_stepchild,pc_child_inlaw,pc_grandchild,pc_parent,pc_parent_inlaw,pc_sibling,pc_sibling_inlaw,pc_fam_other,pc_nonfam_other,total_children,total_adults,total_seniors,total_members,mean_edu,bedrooms,overcrowding,house_own,house_mortgage,house_rent,house_prec,house_other,has_computer,has_television,has_mobileph,total_mobileph,reg_central,reg_chorotega,reg_paccentral,reg_brunca,reg_huetaratl,reg_huetarnor,zone_urban,zone_rural,age,target,dis_cnt,mean_age
0,001ff74ca,ID_654683e33,0.0,0,6,0,1,1,1,0,0,0,1,1,2,1,1,2,2,16,2,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,2,16.0,4,0.500000,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,38,4,0,19.000000
1,003123ec2,ID_5bef9b553,0.0,0,3,0,1,1,0,2,1,3,0,1,1,2,2,4,4,6,4,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,2,0,4,6.5,2,2.000000,0,0,0,0,1,0,0,1,2,0,0,0,0,1,0,1,0,24,2,0,12.750000
2,4616164,ID_ca701e058,0.0,0,4,0,1,1,0,0,2,2,0,0,0,0,2,2,2,3,2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,2,3.0,3,0.666667,0,0,0,0,1,0,0,1,2,0,1,0,0,0,0,0,1,50,2,0,33.000000
3,4983866,ID_1e320689c,0.0,0,5,0,1,1,0,0,1,1,0,1,1,0,2,2,2,8,2,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,2,8.0,2,1.000000,1,0,0,0,0,0,0,1,2,0,0,1,0,0,0,1,0,59,3,0,37.500000
4,5905417,ID_308a05a16,0.0,0,8,0,1,0,0,1,1,2,0,1,1,1,2,3,3,9,3,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,1,0,3,9.0,6,0.500000,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,1,32,2,0,17.333333


In [107]:
train_data_clean.set_index("Unnamed: 0", inplace=True)
del train_data_clean.index.name

KeyError: "None of ['Unnamed: 0'] are in the columns"

In [108]:
train_data_clean.head()

,household_id,id,rent_monthly,bedrooms_overcrowding,rooms,rooms_overcrowding,has_toilet,has_fridge,tablets_owned_count,males_under_12,males_over_12,males_total,females_under_12,females_over_12,females_total,persons_under_12,persons_over_12,persons_total,total_household,schooling,household_size,walls_brick,walls_socket,walls_prefab,walls_waste,walls_wood,walls_zinc,walls_natural,walls_other,floor_tile,floor_cement,floor_other,floor_natural,floor_na,floor_wood,roof_metal,roof_cement,roof_natural,roof_other,has_cieling,water_inside,water_outside,water_na,elec_public,elec_private,elec_na,elec_coop,toilet_na,toilet_sewer,toilet_septank,toilet_latrine,toilet_other,energ_na,energ_cook_elec,energ_cook_gas,energ_cook_wood,trash_truck,trash_buried,trash_burn,trash_dump,trash_water,trash_other,qualwall_bad,qualwall_reg,qualwall_good,qualroof_bad,qualroof_reg,qualroof_good,qualfloor_bad,qualfloor_reg,qualfloor_good,male,female,ec_under_10,ec_free_union,ec_married,ec_divorced,ec_separated,ec_widow/er,ec_single,pc_house_head,pc_spouse,pc_child,pc_stepchild,pc_child_inlaw,pc_grandchild,pc_parent,pc_parent_inlaw,pc_sibling,pc_sibling_inlaw,pc_fam_other,pc_nonfam_other,total_children,total_adults,total_seniors,total_members,mean_edu,bedrooms,overcrowding,house_own,house_mortgage,house_rent,house_prec,house_other,has_computer,has_television,has_mobileph,total_mobileph,reg_central,reg_chorotega,reg_paccentral,reg_brunca,reg_huetaratl,reg_huetarnor,zone_urban,zone_rural,age,target,dis_cnt,mean_age
0,001ff74ca,ID_654683e33,0.0,0,6,0,1,1,1,0,0,0,1,1,2,1,1,2,2,16,2,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,2,16.0,4,0.500000,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,38,4,0,19.000000
1,003123ec2,ID_5bef9b553,0.0,0,3,0,1,1,0,2,1,3,0,1,1,2,2,4,4,6,4,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,2,0,4,6.5,2,2.000000,0,0,0,0,1,0,0,1,2,0,0,0,0,1,0,1,0,24,2,0,12.750000
2,4616164,ID_ca701e058,0.0,0,4,0,1,1,0,0,2,2,0,0,0,0,2,2,2,3,2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,2,3.0,3,0.666667,0,0,0,0,1,0,0,1,2,0,1,0,0,0,0,0,1,50,2,0,33.000000
3,4983866,ID_1e320689c,0.0,0,5,0,1,1,0,0,1,1,0,1,1,0,2,2,2,8,2,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,2,8.0,2,1.000000,1,0,0,0,0,0,0,1,2,0,0,1,0,0,0,1,0,59,3,0,37.500000
4,5905417,ID_308a05a16,0.0,0,8,0,1,0,0,1,1,2,0,1,1,1,2,3,3,9,3,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,1,0,3,9.0,6,0.500000,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,1,32,2,0,17.333333


In [109]:
train_data_clean.shape

(2973, 120)

In [110]:
#Create Indicators/labels
train_labels_testing = np.array(list(train_data_clean[train_data_clean['target'].notnull()]['target'].astype(np.uint8)))
train_labels_testing.shape

(2973,)

In [111]:
#Create Training data predictors set, drop strings / labels
train_pred_testing = train_data_clean[train_data_clean['target'].notnull()].drop(columns = ['household_id', 'id', 'target'])
train_pred_testing.shape

(2973, 117)

In [112]:
#df1 = train_pred[train_pred.isna().any(axis=1)]

In [113]:
#eda_nans(df1).transpose()

In [114]:
# visualization for imbalance in target variable

In [115]:
#Scale Data

features = list(train_pred_testing.columns)

pipeline = Pipeline([('imputer', SimpleImputer(strategy = 'median')), 
                      ('scaler', MinMaxScaler())])

train_pred_testing = pipeline.fit_transform(train_pred_testing)

In [118]:
#balancing the dataset
print('Original dataset shape %s' % Counter(train_labels_testing))

sm = SMOTE(random_state=42)

X_res_testing, y_res_testing = sm.fit_resample(train_pred_testing, train_labels_testing)

print('Resampled dataset shape %s' % Counter(y_res_testing))

Original dataset shape Counter({4: 1954, 2: 442, 3: 355, 1: 222})
Resampled dataset shape Counter({4: 1954, 2: 1954, 3: 1954, 1: 1954})


In [119]:
#Function to pass different model types and perform 10 fold cross validation and hyperparameter tuning

def cv_hyptun_model(train, train_labels, model, name, params, model_results_balanced=None):
    """
    Perform 10 fold cross validation of a model and hyperparameter tuning
    Returns the model performance metrics (stored in model_results_balanced) and the best model (stored in hyptun_best_model)
    
    Pass the following input arguments to the function:
    train -        predictors from the training dataset
    train_labels - labels (y) from the training dataset
    model -        model function to create the model ex. RandomForestClassifier()
    name -         name of the model ex. 'RF'
    params -       a dictionary of parameters you want to tune and the values for 
                   each of those from which you want to perfrom the randomized search
                   ex. params = {
                                # randomly sample numbers from 4 to 204 estimators
                                'n_estimators': randint(4,200),
                                # normally distributed max_features, with mean .25 stddev 0.1, bounded between 0 and 1
                                'max_features': truncnorm(a=0, b=1, loc=0.25, scale=0.1),
                                # uniform distribution from 0.01 to 0.2 (0.01 + 0.199)
                                'min_samples_split': uniform(0.01, 0.199)
                                }
    model_results_balanced is just an empty dataframe to store the model performance metrics
    
    Please note: We have calculated the Balanced Accuracy here even though the input maybe not be a imbalanced dataset
    
    """
    
    model_results_balanced = pd.DataFrame(columns = ['model', 'F1_score', 'Accuracy', 'Accuracy_balanced', 'ROC_AUC'])
    
    # Creating the RandomizedSearchCV object which will be used to perform the CV and hyperparameter tuning 
    cv_hyptun = RandomizedSearchCV(estimator = model, param_distributions = params, n_iter = 1, cv = 10, n_jobs = -1)
    
    # Fitting the model with the predictors and labels
    model_fit = cv_hyptun.fit(train, train_labels)
    
    # Obtianing the parameters for the best model obtained as a result of hyperparamter tuning
    hyptun_best_model = model_fit.best_estimator_.get_params()
    
    # Obtaining the predictions for each observation of the train dataset
    # This will automatically use the best model
    predictions_prob = model_fit.predict_proba(train)
    predictions_labels = model_fit.predict(train)
    
    # Calculating the various model performance metrics
    F1_score = f1_score(train_labels, predictions_labels, average="macro")
    Accuracy = accuracy_score(train_labels, predictions_labels)
    Accuracy_balanced = balanced_accuracy_score(train_labels, predictions_labels)
    ROC_AUC = roc_auc_score(train_labels, predictions_prob, multi_class = "ovo")
    
    # storing model performance metrics in one dataframe
    model_results_balanced = model_results_balanced.append(pd.DataFrame({'model': name, 'F1_score': F1_score,\
                                                                         'Accuracy': Accuracy, 'Accuracy_balanced': Accuracy_balanced,\
                                                                         'ROC_AUC': ROC_AUC},\
                                                                        index = [0]), ignore_index = True)

    return model_results_balanced, hyptun_best_model

In [99]:
X_res = pd.read_csv('kmeans_pca.csv')
y_res = pd.read_csv('train_lab.csv')

X_res['0.1'] = X_res['0.1'].astype(object)

X_res = X_res.to_numpy()
y_res = y_res['0'].to_numpy()

In [101]:
# Random Forest 
#model_params_RF = {'n_estimators': randint(4,200), 'min_samples_split': randint(2,100)}
#RF_model_metrics, RF_best_model = cv_hyptun_model(X_res, y_res, RandomForestClassifier(random_state=42),'RF', params = model_params_RF)
#RF_model_metrics

In [102]:
model_params_RF = {'n_estimators': randint(75,76), 'min_samples_split': randint(19,20)}
RF_model_metrics, RF_best_model = cv_hyptun_model(X_res, y_res, RandomForestClassifier(random_state=42),'RF', params = model_params_RF)
RF_model_metrics

TypeError: roc_auc_score() got an unexpected keyword argument 'multi_class'

In [122]:
typeof(y_res_testing)

[4 2 2 ... 3 3 3]


In [123]:
typeof(y_res)

[4 2 2 ... 3 3 3]


In [124]:
#balanced K Nearest Neighbors

model_params_KNN = {'n_neighbors':randint(2,30)}
KNN_model_metrics, KNN_best_model = cv_hyptun_model(X_res_testing, y_res_testing, KNeighborsClassifier(),'KNN', params = model_params_KNN)
KNN_model_metrics

TypeError: roc_auc_score() got an unexpected keyword argument 'multi_class'

In [29]:
print(X_res)

[[ 0.000e+00 -4.649e-01 -3.169e-01 ...  1.843e-01 -8.125e-02  1.000e+00]
 [ 1.000e+00  1.560e+00 -4.327e-01 ... -4.964e-02 -2.669e-02  0.000e+00]
 [ 2.000e+00  1.956e+00 -4.622e-01 ...  5.759e-01 -1.154e-01  0.000e+00]
 ...
 [ 2.970e+03  2.000e-01 -1.681e+00 ... -1.302e-01 -9.950e-02  1.000e+00]
 [ 2.971e+03  1.902e+00 -1.015e+00 ... -2.406e-01  5.464e-01  0.000e+00]
 [ 2.972e+03  1.019e+00  1.085e+00 ... -1.368e-01 -5.715e-02  0.000e+00]]


In [30]:
# LDA

model_params_LDA = {'n_components':randint(4,120)}
LDA_model_metrics, LDA_best_model = cv_hyptun_model(X_res, y_res, LinearDiscriminantAnalysis(),'LDA', params = model_params_LDA)
LDA_model_metrics

C:\Users\aarti\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(117, 4 - 1) = 3 components.
  ChangedBehaviorWarning)
C:\Users\aarti\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:469: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


,model,F1_score,Accuracy,Accuracy_balanced,ROC_AUC
0,LDA,0.547952,0.547595,0.547595,0.791798
